<a href="https://colab.research.google.com/github/femIA2020/FEM_IA/blob/master/Preprocesamiento_extraccion_de_sentencias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proceso para extraciòn de setencias judiciales



### Instalaciòn de librerias para procesamiento de pdf.

In [5]:
!pip install pyenchant
!apt install enchant
!sudo apt-get install myspell-es
!pip install unidecode
!python -m spacy download es_core_news_sm

     |████████████████████████████████| 61kB 3.1MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 14 not upgraded.
Need to get 1,310 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libtext-iconv-perl amd64 1.7-5build6 [13.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libaspe

## habilitar carpeta de google drive



In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
import nltk.data   
spanish_tokenizer = nltk.data.load('tokenizers/punkt/PY3/spanish.pickle')
from nltk.corpus import stopwords
import string 
from collections import Counter
from _collections import OrderedDict
stop_words= set (stopwords.words('spanish'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
import enchant
d = enchant.Dict("es_US")

##Procesamiento para convertir pdf a imagenes

### Instalaciòn de librererias para convertir pdf a imagen

In [9]:
pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.6-py2.py3-none-any.whl size=13629 sha256=188860f489b870116a776d41741e9b009699d69581b15ac572283582454e5359
  Stored in directory: /root/.cache/pip/wheels/ee/71/72/b98430261d849ae631e283dfc7ccb456a3fb2ed2205714b63f
Successfully built pytesseract


In [10]:
pip install pdf2image

In [11]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.10 [154 kB]
Fetched 154 kB in 1s (123 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 145256 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.10_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.10) ...
Setting up poppler-utils (0.62.0-2ubuntu2.10) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [12]:
!sudo apt install tesseract-ocr 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 14 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 3s (1,777 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

###Proceso para ejecutar la conversion de archivos pdf a imagenes y guradarlos en ruta

In [13]:
from PIL import Image
import pytesseract
import os
import sys
from pdf2image import convert_from_path, convert_from_bytes, pdfinfo_from_path
from shutil import copyfile, copy

In [14]:
# Carpetas donde se encuentran las sentencias, imagenes y txt respectivamente
dir_sent = '/content/drive/MyDrive/Colab Notebooks/saturday ia/justicia abierta/0 versiones finales/'

dir_imag = "/content/drive/MyDrive/Colab Notebooks/saturday ia/justicia abierta/imagenes_pdf/"

dir_txt = "/content/drive/MyDrive/Colab Notebooks/saturday ia/justicia abierta/sentencias_txt/"

# Se crea una instancia de la carpeta
local_download_path = os.path.expanduser(dir_sent)

# Se itera sobre la carpeta
for file in os.listdir(local_download_path):
  if not file[6] == "$" and file.endswith("pdf"):
    print("Processing file: " + file)

    filename = dir_sent + file

    pages = convert_from_bytes(open(filename, "rb").read(), dpi=500, fmt="jpeg")

    image_counter = 1
    # Convertimos a imagenes
    for page in pages:
      filename = dir_imag + file[0:3] +"_page_"+str(image_counter)+".jpg"
      page.save(filename, 'JPEG') 
      image_counter += 1

    filelimit = image_counter-1

    outfile = dir_txt + file[0:3] + ".txt"

    f = open(outfile, "a")

    # Extraemos el texto y guardamos
    for i in range(1, filelimit +1):
      filename = dir_imag + file[0:3] +"_page_"+str(i)+".jpg"
      text = str(((pytesseract.image_to_string(Image.open(filename)))))
      text = text.replace('-\n', '')
      f.write(text) 

    f.close()
    
    # Renombramos el archivo con $ para distinguir

    new_name = file[0:5] + " $ " + file[6:]

    os.rename(dir_sent + file, dir_sent + new_name)
    print("The file " + file + "has finished")

Processing file: 020 a Sent. 77-2017 APIS.pdf


KeyboardInterrupt: ignored

### Proceso para convertir archivos en pdf en imagenes

In [ ]:
local_download_path = os.path.expanduser('/content/drive/My Drive/SaturdaysAI/Sentencias')

for file in os.listdir(local_download_path):
  if file.endswith("pdf"):    
    
    filelimit = image_counter-1

    outfile = "/content/drive/My Drive/SaturdaysAI/txt/" + file[0:3] + ".txt"

    f = open(outfile, "a")

    for i in range(1, filelimit +1):
      filename = "/content/drive/My Drive/SaturdaysAI/imagenes/" + file[0:3] +"_page_"+str(i)+".jpg"
      text = str(((pytesseract.image_to_string(Image.open(filename)))))
      text = text.replace('-\n', '')
      f.write(text) 

    print("The file" + file[0:3]+ "finish successfully")

    f.close()

**Instalar librerias para procesar formatos de *microsfot world* a txt**

In [8]:
!pip install docx2txt


  Created wheel for docx2txt: filename=docx2txt-0.8-cp36-none-any.whl size=3965 sha256=893a38dc1c44de558fbbb7b2f288810359e82d8071c51fec0e684e3ef3bbdd94
  Stored in directory: /root/.cache/pip/wheels/b2/1f/26/a051209bbb77fc6bcfae2bb7e01fa0ff941b82292ab084d596
Successfully built docx2txt


In [9]:
import docx2txt

In [15]:
ruta_origen="/content/drive/My Drive/0_versiones_finales/"
ruta_salida_nueva='/content/drive/My Drive/sentencias_txt/'

In [16]:
local_download_path = os.path.expanduser(ruta_origen)
local_download_path


'/content/drive/My Drive/0_versiones_finales/'

### Proceso para convertir documentos en formato .docx o .doc a   txt

In [14]:
print (local_download_path)
fs=[]
for file in os.listdir(local_download_path):
   
    if (file.endswith("docx") or file.endswith("doc")) and file[4]=='a':
      try:
        filename= ruta_origen+file
        #print (filename)
        result = docx2txt.process(filename)
        archivo_salida=ruta_salida_nueva+file[0:3]+".txt"
        #print (archivo_salida)
        f = open(archivo_salida,"a")
        f.write(result)
        print (" archivo {} se ha procesado de forma exitosa".format(file[0:3]+".txt"))
      except:
        print("Fallo en procesar {}".format(file[0:3]+".txt")) 
     

/content/drive/My Drive/0_versiones_finales/
 archivo 004.txt se ha procesado de forma exitosa
 archivo 005.txt se ha procesado de forma exitosa
 archivo 006.txt se ha procesado de forma exitosa
 archivo 007.txt se ha procesado de forma exitosa
 archivo 012.txt se ha procesado de forma exitosa
 archivo 014.txt se ha procesado de forma exitosa
 archivo 015.txt se ha procesado de forma exitosa
Fallo en procesar 016.txt
 archivo 017.txt se ha procesado de forma exitosa
Fallo en procesar 023.txt
 archivo 025.txt se ha procesado de forma exitosa
 archivo 027.txt se ha procesado de forma exitosa
 archivo 022.txt se ha procesado de forma exitosa
 archivo 024.txt se ha procesado de forma exitosa
 archivo 031.txt se ha procesado de forma exitosa
 archivo 026.txt se ha procesado de forma exitosa
 archivo 028.txt se ha procesado de forma exitosa
 archivo 032.txt se ha procesado de forma exitosa
 archivo 034.txt se ha procesado de forma exitosa
Fallo en procesar 037.txt
 archivo 043.txt se ha proc

##Procesamiento para limpar y aplicar filtros de setencias.


In [ ]:
procesa_setencias()

/content/drive/My Drive/Colab Notebooks/saturday ia/justicia abierta/0 versiones finales/ /content/drive/My Drive/sentencias_txt/ /content/drive/My Drive/Colab Notebooks/saturday ia/justicia abierta/archivos_limpios/
/content/drive/My Drive/sentencias_txt/
/content/drive/My Drive/sentencias_txt/001.txt
/content/drive/My Drive/sentencias_txt/008.txt
/content/drive/My Drive/sentencias_txt/009.txt
/content/drive/My Drive/sentencias_txt/013.txt
/content/drive/My Drive/sentencias_txt/003.txt
/content/drive/My Drive/sentencias_txt/018.txt
/content/drive/My Drive/sentencias_txt/010.txt
/content/drive/My Drive/sentencias_txt/011.txt
/content/drive/My Drive/sentencias_txt/020.txt
/content/drive/My Drive/sentencias_txt/019.txt
/content/drive/My Drive/sentencias_txt/021.txt
/content/drive/My Drive/sentencias_txt/029.txt
/content/drive/My Drive/sentencias_txt/030.txt
/content/drive/My Drive/sentencias_txt/033.txt
/content/drive/My Drive/sentencias_txt/036.txt
/content/drive/My Drive/sentencias_txt

OSError: ignored

In [18]:
def procesa_setencias():
  ruta="/content/drive/My Drive/Colab Notebooks/saturday ia/justicia abierta/0 versiones finales/" #path con sentencias originales
  ruta_salida = "/content/drive/My Drive/sentencias_txt/" #path con  sentencias en txt
  ruta_filtros='/content/drive/My Drive/Colab Notebooks/saturday ia/justicia abierta/archivos_limpios/' #path con sentencias archivos aplicando filtros
  print (ruta,ruta_salida,ruta_filtros)
  limpiar_sentencias(ruta_salida,ruta_filtros)


In [19]:
def limpiar_sentencias(ruta_txt,ruta_filtro):
  local_download_path = os.path.expanduser(ruta_txt)
  print (local_download_path)
  for file in os.listdir(local_download_path):
    print (ruta_txt+file)
    f=open(ruta_txt+file)    
    sentencia=f.read()
   
    f.close()
    f2=open(ruta_filtro+file,"w") # crear archivo donde se guardan las setencias
    word_tokens= word_tokenize(sentencia)
    #word_tokens= suggestions(word_tokens)
    word_tokens = list(filter (lambda token: filtro_1))
    filtro=[]

    for palabra in word_tokens:
      if palabra.lower() not in stop_words:
        filtro.append(palabra.lower())
        f2.write(palabra.lower())
        f2.write(" ")
    word_tokens=[]
    f2.close()  
    archivo_ranking=ruta_filtro+"rank"+file
    guardar_ranking(filtro,archivo_ranking)
  

    
    #todo guardar pandas dataset

In [2]:
def guardar_ranking(filtro,archivo_ranking):
  c= Counter(filtro)
  y=OrderedDict(c.most_common(200))
  with open(archivo_ranking,'w') as f:
    for k,v in y.items():
      f.write(f'{k}: {v} \n')

In [ ]:
###Funcion para aplicar sugerencias con palabras que no tienen significado para la setencias

In [16]:

def suggestions(words, list_word=[]):
  for i in range(0, len(words)):
    if d.check(words[i]) :
      list_word.append(words[i])
    else:
      if len(words[i]) > 1 and (len(d.suggest(words[i])) != 0 ):
        # print(i)
        list_word.append(d.suggest(words[i])[0])

  return list_word

## Procesamieanto para generar dataset

In [26]:
ruta_filtros='/content/drive/My Drive/Colab Notebooks/saturday ia/justicia abierta/archivos_limpios/' #path con sentencias archivos aplicando filtros

import pandas

excel_data_df = pandas.read_excel('/content/drive/My Drive/Colab Notebooks/saturday ia/justicia abierta/result_143_sentencias.xlsx', sheet_name='BASE COMPLETA',
                                  usecols=['Índice','Nombre','ASUNTO/TEMA','INSTANCIA','PUNTUACIÓN'], index_col=0)
print(excel_data_df.shape)


(143, 4)


In [25]:
excel_data_df.describe()

,PUNTUACIÓN
count,143.000000
mean,1.673846
std,1.552542
min,0.000000
25%,0.620000
50%,1.080000
75%,2.000000
max,7.400000


## Genereacion del dataset

## Generamos el dataset  con las salidad de las setencias que se le aplico


*   Eliminacion de stop worlds,
*   Eliminanciòn de puntaciones.
*   Eliminacion de ruido.
*   Eliminacion de  palabras sin relevancia para 





In [27]:
corpus=[]
local_download_path = os.path.expanduser(ruta_filtros)
for file in os.listdir(local_download_path):
  if (file[0:3]!="ran" and file.endswith("txt")):
   
    filename= ruta_filtros+file
    f=open(filename)
    #print (filename)
    corpus_alpha=f.read()
    excel_data_df.at[int(file[0:3]),'CORPUS']=corpus_alpha

print(len(corpus))

0


excel_data_df

In [ ]:
excel_data_df["PUNTUACIÓN"][10:15]>1.08

Índice
11    False
12     True
13    False
14    False
15    False
Name: PUNTUACIÓN, dtype: bool

#### Generar nueva columna para tener clasificaciòn binaria

In [30]:
excel_data_df["score"] = excel_data_df["PUNTUACIÓN"].map(lambda x :1 if (x >1.08) else 0)

In [29]:
excel_data_df.to_csv (r'/content/drive/My Drive/Colab Notebooks/saturday ia/justicia abierta/csv/sentencias_dataframe.csv', index = False, header=True)


In [ ]:

excel_data_df.to_csv (r'/content/drive/My Drive/Colab Notebooks/saturday ia/justicia abierta/csv/sentencias_dataframe.csv', index = False, header=True)


     Índice  ...                      INSTANCIA
0         1  ...              Primera Instancia
1         2  ...              Primera instancia
2         3  ...              Primera Instancia
3         4  ...              Segunda Instancia
4         5  ...  Segunda Instancia (apelación)
..      ...  ...                            ...
138     139  ...              Primera Instancia
139     140  ...              Primera Instancia
140     141  ...              Primera instancia
141     142  ...              Primera instancia
142     143  ...              Primera Instancia

[143 rows x 5 columns]
